In [1]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle
import random
import pdb
from torch.utils.data import DataLoader
import logging
import itertools
import unicodedata
import string
import re
import random
import argparse
from torch import optim
import time
import os
from bleu_score import BLEU_SCORE
# from models_viet import EncoderRNN, AttentionDecoderRNN, DecoderRNN
from load_dataset_viet import *
# from define_training_viet import *
# import torchtext
from torch.utils.data import BatchSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import Sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
%matplotlib inline
import math
import copy
import seaborn

In [2]:
device = torch.device('cuda')

In [3]:
class Vietnamese(Dataset):
    def __init__(self, df, val = False):
        self.df = df
        self.val = val
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        english = self.df.iloc[idx,:]['en_idized']
        viet = self.df.iloc[idx,:]['vi_idized']
        en_len = self.df.iloc[idx,:]['en_len']
        vi_len = self.df.iloc[idx,:]['vi_len']
        if self.val:
            en_data = self.df.iloc[idx,:]['en_data'].lower()
            return [viet,english,vi_len,en_len,en_data]
        else:
            return [viet,english,vi_len,en_len]
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
def vocab_collate_func(batch):
    MAX_LEN_EN = 48
    MAX_LEN_VI = 48
    en_data = []
    vi_data = []
    en_len = []
    vi_len = []
    for datum in batch:
        en_len.append(datum[3])
        vi_len.append(datum[2])
    max_batch_length_en = max(en_len)
    max_batch_length_vi = max(vi_len)
    if max_batch_length_en < MAX_LEN_EN:
        MAX_LEN_EN = max_batch_length_en
    if max_batch_length_vi < MAX_LEN_VI:
        MAX_LEN_VI = max_batch_length_vi
    # padding
    for datum in batch:
        if datum[2]>MAX_LEN_VI:
            padded_vec_s1 = np.array(datum[0])[:MAX_LEN_VI]
        else:
            padded_vec_s1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_LEN_VI - datum[2])),
                                mode="constant", constant_values=PAD_IDX)
        if datum[3]>MAX_LEN_EN:
            padded_vec_s2 = np.array(datum[1])[:MAX_LEN_EN]
        else:
            padded_vec_s2 = np.pad(np.array(datum[1]),
                                pad_width=((0,MAX_LEN_EN - datum[3])),
                                mode="constant", constant_values=PAD_IDX)
        en_data.append(padded_vec_s2)
        vi_data.append(padded_vec_s1)
    vi_data = np.array(vi_data)
    en_data = np.array(en_data)
    vi_len = np.array(vi_len)
    en_len = np.array(en_len)
#     sorted_vi_len = np.argsort(-vi_len)
#     vi_data = vi_data[sorted_vi_len]
#     en_data = en_data[sorted_vi_len]
#     vi_len = vi_len[sorted_vi_len]
#     en_len = en_len[sorted_vi_len]
#     print(en_len)
    vi_len[vi_len>MAX_LEN_VI] = MAX_LEN_VI
    en_len[en_len>MAX_LEN_EN] = MAX_LEN_EN
        
    return [torch.from_numpy(vi_data), torch.from_numpy(en_data),
            torch.from_numpy(vi_len), torch.from_numpy(en_len)]

def convert_idx_2_sent(tensor, lang_obj):
    word_list = []
    for i in tensor:
        if i.item() not in set([PAD_IDX,EOS_token,SOS_token]):
            word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

def convert_id_list_2_sent(list_idx, lang_obj):
    word_list = []
    if type(list_idx) == list:
        for i in list_idx:
            if i not in set([EOS_token]):
                word_list.append(lang_obj.index2word[i])
    else:
        for i in list_idx:
            if i.item() not in set([EOS_token,SOS_token,PAD_IDX]):
                word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

In [4]:
def vocab_collate_func_val(batch):
    return [torch.from_numpy(np.array(batch[0][0])).unsqueeze(0), torch.from_numpy(np.array(batch[0][1])).unsqueeze(0),
            torch.from_numpy(np.array(batch[0][2])).unsqueeze(0), torch.from_numpy(np.array(batch[0][3])).unsqueeze(0),batch[0][4]]

In [5]:
# MAX_LEN = 48
train,val,test,en_lang,vi_lang = train_val_load(48, "", '../../machine_translation')

In [6]:
def shuffle_sorted_batches(train, bs):
    batch_samp_list = list(BatchSampler(SequentialSampler(train), bs, drop_last = False))
    np.random.shuffle(batch_samp_list)
    batch_samp_list_merged = list(itertools.chain(*batch_samp_list))
    return train.iloc[batch_samp_list_merged,:]

In [7]:
bs_dict = {'train':64,'validate':1, 'train_val':1,'val_train':64, 'test':1}
shuffle_dict = {'train':True,'validate':False, 'train_val':False,'val_train':True, 'test':False}
# train_used = shuffle_sorted_batches(train_sorted, bs_dict['train'])
# train_used = train.iloc[:50]
train_used = train.sample(n=100)
val_used = val.sample(n=50)
# val_used = val.iloc[:20]
collate_fn_dict = {'train':vocab_collate_func, 'validate':vocab_collate_func_val,\
                   'train_val':vocab_collate_func_val,'val_train':vocab_collate_func,'test': vocab_collate_func_val}
transformed_dataset = {'train': Vietnamese(train_used),
                       'validate': Vietnamese(val_used, val = True),
                       'train_val':Vietnamese(train.iloc[:50], val = True),
                       'val_train':Vietnamese(val_used),
                       'test':Vietnamese(test, val= True)
                                               }

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs_dict[x], collate_fn=collate_fn_dict[x],
                    shuffle=shuffle_dict[x], num_workers=0) for x in ['train', 'validate', 'train_val','val_train', 'test']}

In [54]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size,n_layers, rnn_type = 'lstm', device = 'cuda'):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = Embedding(input_size, embed_dim, PAD_IDX)
        self.rnn_type =  rnn_type
        self.dropout_in = nn.Dropout(p = 0.1)
        self.n_layers = n_layers
        self.device = device
        if rnn_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_size,batch_first=True,bidirectional=True, num_layers = self.n_layers, dropout = 0.2)
        elif rnn_type == 'lstm':
            self.rnn = LSTM(embed_dim, hidden_size, batch_first=True,bidirectional=True, num_layers = n_layers,dropout = 0.2)

    def forward(self, enc_inp, src_len):
        sorted_idx = torch.sort(src_len, descending=True)[1]
        orig_idx = torch.sort(sorted_idx)[1]
        embedded = self.embedding(enc_inp)
        bs = embedded.size(0)
        output = self.dropout_in(embedded)
        if self.rnn_type == 'gru':
            hidden =  self.initHidden(bs)
            sorted_output = output[sorted_idx]
            sorted_len = src_len[sorted_idx]
            packed_output = nn.utils.rnn.pack_padded_sequence(sorted_output, sorted_len.data.tolist(), batch_first = True)
            packed_outs, hiddden = self.rnn(packed_output,(hidden, c))
            hidden = hidden[:,orig_idx,:]
            output, _ = nn.utils.rnn.pad_packed_sequence(packed_outs, padding_value=PAD_IDX, batch_first = True)
            output = output[orig_idx]
            hidden = hidden.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
            return output, hidden, hidden
        elif self.rnn_type == 'lstm':
            hidden, c = self.initHidden(bs)
            sorted_output = output[sorted_idx]
            sorted_len = src_len[sorted_idx]
            packed_output = nn.utils.rnn.pack_padded_sequence(sorted_output, sorted_len.data.tolist(), batch_first = True)
            packed_outs, (hiddden, c) = self.rnn(packed_output,(hidden, c))
            hidden = hidden[:,orig_idx,:]
            c = c[:,orig_idx,:]
            output, _ = nn.utils.rnn.pad_packed_sequence(packed_outs, padding_value=PAD_IDX, batch_first = True)
            output = output[orig_idx]
            c = c.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
            hidden = hidden.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
            return output, hidden, c
        
    def initHidden(self,bs):
        if self.rnn_type == 'gru' :
            return torch.zeros(self.n_layers*2, bs, self.hidden_size).to(self.device)
        elif self.rnn_type == 'lstm':
            return torch.zeros(self.n_layers*2,bs,self.hidden_size).to(self.device),torch.zeros(self.n_layers*2,bs,self.hidden_size).to(self.device)

class Attention_Module(nn.Module):
    def __init__(self, hidden_dim, output_dim, device = 'cuda'):
        super(Attention_Module, self).__init__()
        self.l1 = Linear(hidden_dim, output_dim, bias = False)
        self.l2 = Linear(hidden_dim+output_dim, output_dim, bias =  False)
        self.device = device
        
    def forward(self, hidden, encoder_outs, src_lens):
        ''' hiddden: bsz x hidden_dim
        encoder_outs: bsz x sq_len x encoder dim (output_dim)
        src_lens: bsz
        
        x: bsz x output_dim
        attn_score: bsz x sq_len'''
        x = self.l1(hidden)
        att_score = (encoder_outs.transpose(0,1) * x.unsqueeze(0)).sum(dim = 2)
        seq_mask = sequence_mask(src_lens, max_len = max(src_lens).item(), device = self.device).transpose(0,1)
        masked_att = seq_mask*att_score
        masked_att[masked_att==0] = -1e10
        attn_scores = F.softmax(masked_att, dim=0)
        x = (attn_scores.unsqueeze(2) * encoder_outs.transpose(0,1)).sum(dim=0)
        x = torch.tanh(self.l2(torch.cat((x, hidden), dim=1)))
        return x, attn_scores
        
class AttentionDecoderRNN(nn.Module):
    def __init__(self, output_size, embed_dim, hidden_size, n_layers = 1, attention = True, device = 'cuda'):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        encoder_output_size = hidden_size
        self.embedding = Embedding(output_size, embed_dim, PAD_IDX)
        self.dropout = nn.Dropout(p=0.1)
        self.n_layers = n_layers
        self.device = device
        self.att_layer = Attention_Module(self.hidden_size, encoder_output_size,self.device) if attention else None
        self.layers = nn.ModuleList([
            LSTMCell(
                input_size=self.hidden_size + embed_dim if ((layer == 0) and attention) else embed_dim if layer == 0 else hidden_size,
                hidden_size=hidden_size,
            )
            for layer in range(self.n_layers)
        ])
        self.fc_out = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.no_len_control_ll = nn.Linear(1024, 1024)
        self.len_control_ll = nn.Linear(1034, 1024)
        self.len_params = nn.Parameter(torch.rand(1,10))
        
    def forward(self, dec_input, context_vector, prev_hiddens, prev_cs, encoder_outputs, src_len, len_control=False, init=False):
        bsz = dec_input.size(0)
        output = self.embedding(dec_input)
        output = self.dropout(output)
        if init:
            if len_control:
                prev_hiddens = self.len_control_ll(torch.cat((torch.mean(prev_hiddens, dim=1), \
                                                          self.len_params.repeat(bsz, 1)),dim=1)).unsqueeze(0)
            else:
                prev_hiddens = self.no_len_control_ll(torch.mean(prev_hiddens, dim=1)).unsqueeze(0)
            
#         print("decoder", prev_hiddens.size(), len_control)
#         print(dec_input.size(), prev_hiddens.size(), context_vector.size(), prev_cs.size(), len_control, init)
        if self.att_layer is not None:
            cated_input = torch.cat([output.squeeze(1),context_vector], dim = 1)
        else:
            cated_input = output.squeeze(1)
        new_hiddens = []
        new_cs = []
        for i, rnn in enumerate(self.layers):
            hidden, c = rnn(cated_input, (prev_hiddens[i], prev_cs[i]))
            cated_input = self.dropout(hidden)
            new_hiddens.append(hidden.unsqueeze(0))
            new_cs.append(c.unsqueeze(0))
        new_hiddens = torch.cat(new_hiddens, dim = 0)
        new_cs = torch.cat(new_cs, dim = 0)

        # apply attention using the last layer's hidden state
        if self.att_layer is not None:
            out, attn_score = self.att_layer(hidden, encoder_outputs, src_len)
        else:
            out = hidden
            attn_score = None
        context_vec = out
        out = self.dropout(out)
        out_vocab = self.softmax(self.fc_out(out))

        return out_vocab, context_vec, new_hiddens, new_cs, attn_score

In [55]:
def Embedding(num_embeddings, embedding_dim, padding_idx):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.uniform_(m.weight, -0.1, 0.1)
    nn.init.constant_(m.weight[padding_idx], 0)
    return m


def LSTM(input_size, hidden_size, **kwargs):
    m = nn.LSTM(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def LSTMCell(input_size, hidden_size, **kwargs):
    m = nn.LSTMCell(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def Linear(in_features, out_features, bias=True, dropout=0):
    """Linear layer (input: N x T x C)"""
    m = nn.Linear(in_features, out_features, bias=bias)
    m.weight.data.uniform_(-0.1, 0.1)
    if bias:
        m.bias.data.uniform_(-0.1, 0.1)
    return m

In [56]:
def sequence_mask(sequence_length, max_len=None, device = 'cuda'):
    if max_len is None:
        max_len = sequence_length.max().item()
    batch_size = sequence_length.size(0)
    seq_range = torch.arange(0, max_len).long()
    seq_range_expand = seq_range.unsqueeze(0).repeat([batch_size,1])
    seq_range_expand = seq_range_expand.to(device)
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return (seq_range_expand < seq_length_expand).float()

In [57]:
def encode_decode(encoder, decoder, data_en, data_de, src_len, tar_len, rand_num = 0.95, len_cont = False, val = False):
    if not val:
        use_teacher_forcing = True if random.random() < rand_num else False
        bss = data_en.size(0)
        en_out, en_hid, en_c = encoder(data_en, src_len)
        max_src_len_batch = max(src_len).item()
        max_tar_len_batch = max(tar_len).item()
        prev_hiddens = en_out
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
        prev_output = torch.zeros((bss, en_out.size(-1))).to(device)
        init = True
        if use_teacher_forcing:
            d_out = []
            for i in range(max_tar_len_batch):
                out_vocab, prev_output, prev_hiddens, prev_cs, attention_score = decoder(decoder_input, prev_output, \
                                                                                        prev_hiddens, prev_cs, en_out,\
                                                                                        src_len, len_cont, init)
                init = False
                d_out.append(out_vocab.unsqueeze(-1))
                decoder_input = data_de[:,i].view(-1,1)
            d_out = torch.cat(d_out,dim=-1)
        else:
            d_out = []
            for i in range(max_tar_len_batch):
                out_vocab, prev_output, prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                        prev_hiddens,prev_cs, en_out,\
                                                                                        src_len, len_cont, init)
                init = False
                d_out.append(out_vocab.unsqueeze(-1))
                topv, topi = out_vocab.topk(1)
                decoder_input = topi.squeeze().detach().view(-1,1)
            d_out = torch.cat(d_out,dim=-1)
        return d_out
    else:
        encoder.eval()
        decoder.eval()
        bss = data_en.size(0)
        en_out,en_hid,en_c = encoder(data_en, src_len)
        max_src_len_batch = max(src_len).item()
        max_tar_len_batch = max(tar_len).item()
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bss).to(device)
        prev_output = torch.zeros((bss, en_out.size(-1))).to(device)
        d_out = []
        for i in range(max_tar_len_batch):
            out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                    prev_hiddens,prev_cs, en_out,\
                                                                                    src_len)
            d_out.append(out_vocab.unsqueeze(-1))
            topv, topi = out_vocab.topk(1)
            decoder_input = topi.squeeze().detach().view(-1,1)
        d_out = torch.cat(d_out,dim=-1)
        return d_out

In [58]:
def flatten_cel_loss(input,target,nll):
    input = input.transpose(1,2)
    bs, sl = input.size()[:2]
    return nll(input.contiguous().view(bs*sl,-1),target.contiguous().view(-1))

In [32]:
new validation torch.Size([1, 1]) torch.Size([1, 1024]) torch.Size([1, 1, 1024]) torch.Size([1, 1, 1024]) torch.Size([1, 13, 1024])
sec dec torch.Size([1, 1]) torch.Size([1, 1024]) torch.Size([1, 26, 1024]) torch.Size([1, 1, 1024]) torch.Size([1, 26, 1024])

SyntaxError: invalid syntax (<ipython-input-32-94577690276b>, line 1)

In [71]:
def validation_new(encoder, decoder, encoder2, decoder2, val_dataloader, lang_en, lang_vi, m_type, verbose = False, replace_unk = False):
    encoder.eval()
    decoder.eval()
    pred_corpus = []
    true_corpus = []
    src_corpus = []
    running_loss = 0
    running_total = 0
    bl = BLEU_SCORE()
    attention_scores_for_all_val = []
    for data in val_dataloader:
        
        encoder_i = data[1].to(device)
        src_len = data[3].to(device)
        
        bs,sl = encoder_i.size()[:2]
        
        en_out,en_hid,en_c = encoder(encoder_i, src_len)
        
        max_src_len_batch = max(src_len).item()
        
        prev_hiddens = en_hid
        prev_cs = en_c
        decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
        prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
        
        d_out = []
        attention_scores = []
        init = True
#         print("Validation new first decoder")
#         print("new validation", decoder_input.size(), prev_output.size(), prev_hiddens.size(),prev_cs.size(),en_out.size())
        for i in range(sl*2):
            out_vocab, prev_output,prev_hiddens, prev_cs, attention_score = decoder(decoder_input,prev_output, \
                                                                                    prev_hiddens, prev_cs, en_out,\
                                                                                    src_len, False, init)
            init = False
            topv, topi = out_vocab.topk(1)
            d_out.append(topi.item())
            decoder_input = topi.squeeze().detach().view(-1,1)
#             if m_type == 'attention':
#                 attention_scores.append(attention_score.unsqueeze(-1))
            if topi.item() == EOS_token:
                break
        
        d_out = torch.tensor(d_out).view(1,-1).to(device)
        
        encoder_i2 = d_out
        src_len2 = torch.tensor([encoder_i2.size(1)]).to(device)
        bs, s2 = encoder_i2.size()
        
        en_out2, en_hid2, en_c2 = encoder2(encoder_i2, src_len2)
        
        prev_hiddens2 = en_out2
        prev_cs2 = en_c2
        decoder_input2 = torch.tensor([[SOS_token]]*bs).to(device)
        prev_output2 = torch.zeros((bs, en_out2.size(-1))).to(device)
        
        dec2_out = []
        init = True
#         print("validation new second decoder")
#         print("sec dec",decoder_input2.size(), prev_output2.size(), prev_hiddens2.size(), prev_cs2.size(),en_out2.size())
        for i in range(s2*2):
            out_vocab2, prev_output2, prev_hiddens2, prev_cs2, attention_scores2 = decoder(decoder_input2, prev_output2, \
                                                                                     prev_hiddens2, prev_cs2, en_out2, \
                                                                                     src_len2, True, init)
            init = False            
            topv, topi = out_vocab2.topk(1)
            dec2_out.append(topi.item())
            decoder_input2 = topi.squeeze().detach().view(-1,1)
            if topi.item == EOS_token:
                break
                
        d_out = torch.tensor(dec2_out).to(device)
        
        
#         true_sent = convert_id_list_2_sent(data[1][0],lang_en)
#         true_corpus.append(true_sent)
        if replace_unk:
            true_sent = convert_id_list_2_sent(data[1][0],lang_en)
            true_corpus.append(true_sent)
        else:
            true_corpus.append(data[-1])
        src_sent = convert_id_list_2_sent(data[1][0],lang_en)
        src_corpus.append(src_sent)
#         print(d_out)
        pred_sent = convert_id_list_2_sent(d_out,lang_en)
        pred_corpus.append(pred_sent)
#         print("src : ",src_sent)
#         print("Pred : ",pred_sent)
        if m_type == 'attention':
            attention_scores = torch.cat(attention_scores, dim = -1)
            attention_scores_for_all_val.append(attention_scores)
        if verbose:
            print("True Sentence:",data[-1])
            print("Pred Sentence:", pred_sent)
            print('-*'*50)
    score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
    return score, attention_scores_for_all_val, pred_corpus, src_corpus

In [88]:
def train_model_compression(encoder_optimizer, decoder_optimizer, encoder_optimizer2, decoder_optimizer2, \
                            encoder, decoder, encoder2, \
                            decoder2, loss_fun, m_type, dataloader, en_lang, vi_lang,\
                num_epochs=60, val_every = 1, train_bleu_every = 10,clip = 0.1, rm = 0.8, enc_scheduler = None,\
               dec_scheduler = None, enc_scheduler2 = None, dec_scheduler2 = None, enc_dec_fn = encode_decode, val_fn = validation_new):
    best_score = 0
    best_bleu = 0
    loss_hist = {'train': [], 'val_train': []}
    bleu_hist = {'train': [], 'validate': []}
    best_encoder_wts = None
    best_decoder_wts = None
    best_encoder_wts2 = None
    best_decoder_wts2 = None
    phases = ['train']
    for epoch in range(num_epochs):
        for ex, phase in enumerate(phases):
            start = time.time()
            total = 0
            top1_correct = 0
            running_loss = 0
            running_total = 0
            if phase == 'train':
                encoder.train()
                decoder.train()
                encoder2.train()
                decoder2.train()
            else:
                encoder.eval()
                decoder.eval()
                encoder2.eval()
                decoder2.eval()
                
            for data in dataloader[phase]:
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()
                encoder_optimizer2.zero_grad()
                decoder_optimizer2.zero_grad()

                encoder_i = data[1].to(device)
                decoder_i = data[0].to(device)
                src_len = data[3].to(device)
                tar_len = data[2].to(device)
                
                out = enc_dec_fn(encoder, decoder, encoder_i, decoder_i, src_len, tar_len, rand_num = rm, len_cont=False, val = False )
                
                topi = out.argmax(dim=1)
                
                encoder2_i = topi
                decoder2_i = data[1].to(device)
                src_len2 = data[2].to(device)
                tar_len2 = data[3].to(device)
            
#                 print(out.size(),encoder_i.size(), encoder2_i.size(), decoder_i.size(), decoder2_i.size())
                out2 = enc_dec_fn(encoder2, decoder2, encoder2_i, decoder2_i, src_len2, tar_len2, rand_num = rm, len_cont= True, val = False )
                
#                 print("out size = ", out.size(), topi.size(), out2.size())
                N = decoder_i.size(0)
                loss = loss_fun(out.float(), decoder_i.long()) + loss_fun(out2.float(), decoder2_i.long())
                running_loss += loss.item() * N
                
                total += N
                if phase == 'train':
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
                    torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
                    torch.nn.utils.clip_grad_norm_(encoder2.parameters(), clip)
                    torch.nn.utils.clip_grad_norm_(decoder2.parameters(), clip)
                    encoder_optimizer.step()
                    decoder_optimizer.step()
                    encoder_optimizer2.step()
                    decoder_optimizer2.step()
                    
            epoch_loss = running_loss / total 
            loss_hist[phase].append(epoch_loss)
            print("epoch {} {} loss = {}, time = {}".format(epoch, phase, epoch_loss,
                                                                           time.time() - start))
        if (enc_scheduler is not None) and (dec_scheduler is not None):
            enc_scheduler.step(loss_hist['train'][-1])
            dec_scheduler.step(loss_hist['train'][-1])
            enc_scheduler2.step(loss_hist['train'][-1])
            dec_scheduler2.step(loss_hist['train'][-1])
            
        if epoch%val_every == 0:
            val_bleu_score, _,_,_ = val_fn(encoder, decoder, encoder2, decoder2, dataloader['validate'],\
                                            en_lang,vi_lang,m_type, verbose=False, \
                                            replace_unk=True)
            
            bleu_hist['validate'].append(val_bleu_score)
            print("validation BLEU = ", val_bleu_score)
            if val_bleu_score > best_bleu:
                best_bleu = val_bleu_score
                best_encoder_wts = encoder.state_dict()
                best_decoder_wts = decoder.state_dict()
                best_encoder_wts2 = encoder2.state_dict()
                best_decoder_wts2 = decoder2.state_dict()
                
        print('='*50)
    encoder.load_state_dict(best_encoder_wts)
    decoder.load_state_dict(best_decoder_wts)
    encoder2.load_state_dict(best_encoder_wts2)
    decoder2.load_state_dict(best_decoder_wts2)
    
    print("Training completed. Best BLEU is {}".format(best_bleu))
    return encoder, decoder, encoder2, decoder2, loss_hist, bleu_hist

In [89]:
encoder_w_att = EncoderRNN(en_lang.n_words, 512,512, 1).to(device)
encoder_w_att_2 = EncoderRNN(vi_lang.n_words, 512,512, 1).to(device)

decoder_w_att = AttentionDecoderRNN(vi_lang.n_words, 512,1024,n_layers=1, attention = True).to(device)
decoder_w_att_2 = AttentionDecoderRNN(en_lang.n_words, 512,1024,n_layers=1, attention = True).to(device)

/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [91]:
# encoder_optimizer = optim.Adam(encoder_wo_att.parameters(), lr = 5e-3)
# decoder_optimizer = optim.Adam(decoder_wo_att.parameters(), lr = 5e-3)
encoder_optimizer = optim.SGD(encoder_w_att.parameters(), lr=0.1,nesterov=True, momentum = 0.99)
encoder_optimizer2 = optim.SGD(encoder_w_att_2.parameters(), lr=0.1,nesterov=True, momentum = 0.99)

decoder_optimizer = optim.SGD(decoder_w_att.parameters(), lr=0.1,nesterov=True, momentum = 0.99)
decoder_optimizer2 = optim.SGD(decoder_w_att_2.parameters(), lr=0.1,nesterov=True, momentum = 0.99)

enc_scheduler = ReduceLROnPlateau(encoder_optimizer, min_lr=1e-4,factor = 0.5,  patience=0)
enc_scheduler2 = ReduceLROnPlateau(encoder_optimizer2, min_lr=1e-4,factor = 0.5,  patience=0)

dec_scheduler = ReduceLROnPlateau(decoder_optimizer, min_lr=1e-4,factor = 0.5,  patience=0)
dec_scheduler2 = ReduceLROnPlateau(decoder_optimizer2, min_lr=1e-4,factor = 0.5,  patience=0)





# enc_scheduler = None
# enc_scheduler2 = None
# dec_scheduler = None
# dec_scheduler2 = None

In [92]:
criterion = nn.NLLLoss(ignore_index=PAD_IDX)

In [95]:
encoder_w_att, decoder_w_att, encoder_w_att_2, decoder_w_att_2, loss_hist, acc_hist = \
                        train_model_compression(encoder_optimizer, decoder_optimizer, \
                                                            encoder_optimizer2, decoder_optimizer2, encoder_w_att, \
                                                decoder_w_att, encoder_w_att_2, decoder_w_att_2, criterion,\
                                            "non_attention", dataloader,en_lang, vi_lang, num_epochs = 50, rm = 0.95,\
                                           enc_scheduler = enc_scheduler, dec_scheduler = dec_scheduler, \
                                                enc_scheduler2 = enc_scheduler2, dec_scheduler2 = dec_scheduler2)

epoch 0 train loss = 10.609091453552246, time = 2.3843555450439453
validation BLEU =  0.0
epoch 1 train loss = 10.57917121887207, time = 2.3345425128936768
validation BLEU =  0.0
epoch 2 train loss = 10.566456565856933, time = 2.3343799114227295
validation BLEU =  0.0
epoch 3 train loss = 10.542923278808594, time = 2.2836644649505615
validation BLEU =  0.0
epoch 4 train loss = 10.50886402130127, time = 2.3286988735198975
validation BLEU =  0.0
epoch 5 train loss = 10.481978912353515, time = 2.3347620964050293
validation BLEU =  0.0
epoch 6 train loss = 10.462536773681641, time = 2.336409091949463
validation BLEU =  0.0
epoch 7 train loss = 10.462386932373047, time = 2.269564151763916
validation BLEU =  0.0
epoch 8 train loss = 10.428451690673828, time = 2.3299927711486816
validation BLEU =  0.0
epoch 9 train loss = 10.40544075012207, time = 2.3351430892944336
validation BLEU =  0.0
epoch 10 train loss = 10.385713195800781, time = 2.334787130355835
validation BLEU =  0.0
epoch 11 train 

AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
RuntimeError: Input batch size 1 doesn't match hidden[0] batch size 26